In [133]:
!pip install tensorflow-gpu==2.0.0

In [134]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

In [0]:
t = tf.reshape(x_train, (60000,-1))
t = tf.dtypes.cast(t, dtype = tf.float32)

In [0]:
n, m = t.shape
nh1 = 100

In [0]:
w1 = tf.random.normal((m, nh1))*0.0001 
w2 = tf.random.normal((nh1, 10))*0.0001 

In [0]:
def relu(x):
  return tf.math.maximum(x, 0.)

In [0]:
def log_softmax(x): 
  return x - tf.math.log(tf.math.reduce_sum((tf.math.exp(x)), axis=-1, keepdims=True))

In [0]:
x = tf.linalg.matmul(t, w1)
x = relu(x)
x = tf.linalg.matmul(x, w2)
# x = log_softmax(x)

### ¿Cómo entrenar una red neuronal?

You take the gradient of a tensor to help you figure out what you need to do to minimize error. [Cami Williams de Fcebook](https://medium.com/@cwillycs/committing-to-pytorch-by-someone-who-doesnt-know-a-ton-about-pytorch-fa222253cf2d).

Gradient descent is an algorithm that allows us to minimize error efficiently. The error is determined by our data. We have data that is properly classified and improperly classified. We take the gradient to decrease the number of improperly classified items.

Definimos una función objetivo, una función que nos indicará si nuestro modelo mejora o empeora. A esta función le llamaremos función de pérdida y existen muchos tipos diferentes. Para esta explicación, y porque es muy recomendada para imágenes, usamos la función cross-entropy: 

$$ CrossEntropy_i(y, \log p(y)) = -\sum_i y_i\, \log p(y_i) $$

Esta es la formula para calcular la pérdida para una determinada imagen $i$. Coloco como argumento $\log p(y)$ porque nosotros estamos obteniendo el log softmax; ya conocemos los valores $\log p(y)$. Es decir, el resultado de nuestro grafo es $\log p(y)$. Ahora multipliquemos por el valor correcto de nuestras etiquetas y obtengamos el promedio para obtener el "costo".

Notar que nuestra función de pérdida, y por lo tanto la de costo, depende de los valores de nuestros pesos `w1` y `w2` por lo que podemos escribirla así:

$$ CrossEntropy_i(y, \log p(y)(w1, w2)) = -\sum_i y_i\, \log p(y_i) $$

La función de pérdida nos dice la pérdida de un solo ejemplo. La función de costo nos indica la pérdida promedio de todos nuestros ejemplos. 

"Una función de costo básicamente nos dice "qué tan bueno" es nuestro modelo para hacer predicciones para un valor dado de myb." - [Parul Pandley](https://towardsdatascience.com/understanding-the-mathematics-behind-gradient-descent-dde5dc9be06e).

Para explicar el backward pass y cómo optimizamos nuestros modelos, continuemos con nuestro tensor `x` resultante de aplicar la función `softmax`. La forma de `x` es `TensorShape([60000, 10])` donde cada una de las 60,000 imágenes tiene un tensor de rango uno/vector asociado con un largo de 10, un "espacio" para cada categoría.

Para entender la función de pérdida cross entropy miremos dos valores de nuestro tensor rango 1 de labels (etiquetas, en español). Para hacer esto usamos indexing al estilo de numpy. Es muy recomendable esta habilidad, un buen tutorial se encuentra en [geeks for geeks](https://www.geeksforgeeks.org/indexing-in-numpy/). Con `y_train[:2]` indicamos que queremos ver los primeros dos elementos de `y_train`. Al correrlo obtenemos `array([5, 0], dtype=uint8)`. Esto significa que las primeras dos imágenes representan un 5 y un 0. ¿Cuál es la probabilidad que nuestro modelo asigna a que la primera imagen sea un 5 y la segunda un 0? Esto lo sabemos al observar los resultados para nuestras primeras dos imágenes después de pasarlas por nuestro grafo. 

In [145]:
x[:2]

<tf.Tensor: id=590, shape=(2, 10), dtype=float32, numpy=
array([[ 1.6505044e-04,  5.1404742e-05,  2.8658978e-04,  1.5098239e-04,
         2.5018206e-04,  1.9655205e-04, -1.0425002e-04, -1.1724514e-04,
        -3.4099310e-05, -7.8170931e-05],
       [ 1.5946536e-04,  1.8091056e-04,  1.3446536e-04,  1.0787283e-04,
         3.6331557e-04,  3.1268009e-04, -6.9758760e-05, -5.7324309e-05,
         1.1261831e-04,  1.4456987e-04]], dtype=float32)>

Vemos que `x[:2]` nos regresa los valores, por imagen, que asigna nuestro grafo a cada una de las categorías. Por ejemplo, en este caso sabemos que la primera imagen representa un 5, entonces el valor número 5 de la primera imagen debe ser igual a 1, o lo más cercano posible. Podemos acceder a este valor usando indexing: `x[0][4]`. Recordar que en computación comenzamos a contar desde cero por lo que el quinto valor en un grupo ordenado de datos es el cuarto. 

Siguiendo este hilo, veamos los valores que nuestro grafo imprimió para las categorías que les eran correctas. 
*Nota: para hacer un indexing más avanzado conviene cambiar a pytorch o a numpy.

In [146]:
x.numpy()[[0,1],[4,0]]

array([0.00025018, 0.00015947], dtype=float32)

El resultado nos indica que nuestro grafo arroja `array([-2.3026927, -2.302366 ], dtype=float32)` y los valores correctos debieron ser array([1, 1 ], dtype=float32). Pero está bien, aún no entrenamos nuestro modelo.

Entonces, buscamos que las predicciones para las categorías correctas tengan 1s y las categorías erróneas tengan 0s. Ok, con esto podemos definir nuestra función de pérdida cross entropy:

In [0]:
def cross_entropy(pred, etiquetas):
  return tf.convert_to_tensor(-pred.numpy()[range(etiquetas.shape[0]), etiquetas].mean())

No te preocupes si no entiendes el por qué de esta definición a la primera. Experimenta corriendo el código paso por paso para entender cómo funciona. Recuerda que las categorías correctas tendrán un 1. No te estreses demasiado pues tensorflow tiene su propia cross entropy de forma que tú no tengas que definirla desde 0. De hecho, la implementación de tensorflow computa el softmax y el cross entropy a la vez: `tf.nn.softmax_cross_entropy_with_logits`. En lo persona, prefieriro usar lo ya definido.

Computemos nuestra pérdida para cada una de nuestras observaciones. Para esto primero necesitamos que nuestras etiquetas, `y_train`, tengan la forma correcta. 

In [0]:
y_train = tf.reshape(tf.convert_to_tensor(y_train, dtype=tf.float32), shape=(-1,1))

Por eficiencia computacional, tensorflow computa las funciones softmax y cross entropy juntas.

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=x)

El resultado es un tensor de rango uno con forma `TensorShape([60000])`; la pérdida para cada una de nuestras imágenes. Ahora obtengamos la pérdida para nuestro modelo. ¿Cómo? Saquemos promedio.

In [0]:
perdida = tf.reduce_sum(cross_entropy) / (cross_entropy.shape[0])

Obtenemos un tensor rango cero/escalar con la pérdida para nuestro modelo.

¡Nuestro objetivo es hacer que eseta pérdida sea lo menor posible! Ahora estamos prácticamente listos para entrenar nuestro modelo. ¿Cómo lo haremos? Como vemos nuestra función de pérdida depende de nuestras variables. ¿Cuáles son nuestras variables en el grafo que acabamos de definir? Los datos no son pues estos se mantienen constantes, no cambian, son lo que son. La variables son nuestros weights, `w1` y `w2`. Nosotros debemos ajustar nuestros weights para que nuestra pérdida sea lo menor posible.

Observemos un ejemplo gráfico para comprender más esto. Usemos la siguiente figura como ejemplo. Supongamos que $x$ es `w1`, $y$ es `w2`, y $z$ es nuestra pérdida. En otras palabras, la pérdida depende de nuestros dos weights. Moviendo los valores de nuestros weights debemos encontrar el global minima (mínimo global, en español); el punto donde, dados los weights, la pérdida es más baja. Como vemos, la función de pérdida no es convexa ni concava. A veces nos debemos conformar con encontrar el local minima (mínimo local). 


![Yosh Ginsu]({{site.baseurl, :class="img-responsive"}}/assets/img/perdida.png)

La forma de obtener el valor más pequeño posible de la función de pérdida es moviendo los weights en la dirección correcta. Así de intuitivo. A este proceso le llamamos optimización. 

### El algoritmo gradient descent

"El gradient descent es uno de los algoritmos más populares para realizar la optimización y, con mucho, la forma más común de optimizar las redes neuronales. Es un algoritmo de optimización iterativo utilizado para encontrar el valor mínimo para una función." - [Parul Pandley](https://towardsdatascience.com/understanding-the-mathematics-behind-gradient-descent-dde5dc9be06e).

En nuestra figura pasada tenemos que definir hacía dónde nos tenemos que mover. ¿Para arriba? ¿Para abajo? ¿Para la izquierda? o ¿para la derecha? También tomamos en cuenta el tamaño del moviemiento. ¿Pequeño? ¿Grande? ¿Un intermedio?

Para tomar estas decisiones el algoritmo gradient descent nos ayudará utlizando los gradientes de la función de costo con respecto a las variables de las que depende, en este caso `w1` y `w2`. 

Ya sabemos como computar derivadas con tensorflow pero aún no hemos entrado en detalle sobre lo que es el gradiente que necesitamos y la utilidad de las derivadas. 

La diferencia entre los términos derivada y gradiente es borrosa. En ocasiones son usadas como sinónimos. También se dice que [el gradiente es una palabra más "fancy" para derivada](https://betterexplained.com/articles/vector-calculus-understanding-the-gradient/#targetText=The%20gradient%20is%20a%20fancy,a%20function%20(intuition%20on%20why)). En estos textos usaremos la bella definición de [Better Explained](https://betterexplained.com/articles/vector-calculus-understanding-the-gradient/#targetText=The%20gradient%20is%20a%20fancy,a%20function%20(intuition%20on%20why)): "El término "gradiente" se usa típicamente para funciones con varias entradas y una sola salida (un campo escalar). [...] El uso de "gradiente" para funciones de una sola variable es innecesariamente confuso. Mantenlo simple." En otras palabras, usamos el término derivada en funciones de una única entrada, $y = x + 2$ por ejemplo, y gradiente para funciones de múltiples entradas, nuestra función de costo por ejemplo.

Aclarado esto, ¿qué nos dice el gradiente? El gradiente "es un vector (una dirección para la cual moverse) que indica la dirección del mayor aumento de una función, y es cero en un máximo local o mínimo local (porque no hay una única dirección de aumento)" - [Better Explained](https://betterexplained.com/articles/vector-calculus-understanding-the-gradient/#targetText=The%20gradient%20is%20a%20fancy,a%20function%20(intuition%20on%20why)).


Entonces, los gradientes nos ayudarán indicando la dirección en la que nos tenemos que mover si queremos que nuestro costo disminuya.

EL algoritmo de gradient descent se ve así en código para nuestro `w1` (se lo pueden imaginar en notación matématica): `w1 = w1 - (learning_rate * dw1)`. Estamos actualizando `w1`.

En palabras, nuestros weights `w1` ahora son iguales a lo que eran antes menos el producto entre el "learning rate" y `dw1`. `dw1` es el gradiente de nuestra función de costo con respecto a `w1`.

El learning rate (tasa de aprendizaje, en español) es la tasa de cambio con la que actualizamos nuestros pesos. Es el "[...] tamaño de los pasos dados para alcanzar el mínimo [...] Podemos cubrir más áreas con pasos más grandes/mayor learning rate, pero corremos el riesgo de sobrepasar los mínimos. Por otro lado, los pasos pequeños/learning rates más pequeños consumirán mucho tiempo para alcanzar el punto más bajo." - [Better Explained](https://betterexplained.com/articles/vector-calculus-understanding-the-gradient/#targetText=The%20gradient%20is%20a%20fancy,a%20function%20(intuition%20on%20why)).

La siguiente figura obtenida de [Ashwath Salimath](https://medium.com/octavian-ai/how-to-use-the-learning-rate-finder-in-tensorflow-126210de9489) nos permite entender mejor el rol del learning rate.

![Yosh Ginsu]({{site.baseurl, :class="img-responsive"}}/assets/img/learning.png)

La figura de hasta la izquierda muestra como un learning rate muy pequeño puede tardar mucho en encontrar el mínimo de una función; la de hasta la derecho muestra como un learning rate muy grande puede brincar de lado a lado con la posibilidad de jamás encontrar el mínimo. La tasa ideal es la de la figura de en medio, un avance constante y seguro. 


`w2 = w2 - (learning_rate * dw2)` hace lo mismo con respecto a `w2`. Así vamos optimizando; encontrando el mínimo de la función de costo con respecto a nuestros weights.

### Optimizando

Primero indiquemos a tensorflow que queremos que siga la actividad en la que incurren nuestros weights. Vamos a aclarar que son variables y con el argumento `trainable=True` aclaramos que entrenaremos estas variables. En el argumento `initial_value` indicamos cuál será el primer valor que tendrá esta variable, posteriormente cambiará pues lo vamos a entrenar.

In [0]:
w1 = tf.Variable(initial_value = w1, dtype= "float32", trainable=True)
w2 = tf.Variable(initial_value = w2, dtype= "float32", trainable=True)

Definimos el learning rate con el que operaremos.

In [0]:
learning_rate = 0.5

Computamos primero `dw1` y `dw2` utilizando `tf.GradientTape`.  

In [0]:
with tf.GradientTape() as tape:
  x = tf.linalg.matmul(t, w1)
  x = relu(x)
  x = tf.linalg.matmul(x, w2)
  # x = log_softmax(x)
  # perdida = cross_entropy(x, y_train)
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=x)
  costo = tf.reduce_sum(cross_entropy) / (cross_entropy.shape[0])
    
dw1, dw2 = tape.gradient(costo, [w1, w2])

Optimizamos aplicando grandient descent.

In [0]:
w1 = w1 - (learning_rate * dw1)
w2 = w2 - (learning_rate * dw2)

¿ Qué acabamos de hacer? Acabamos de hacer que nuestros weights avancen hacía nuestro minimizar el costo. Podemos repetir este proceso hasta encontrar un mínimo. Éxito. 

### ¿Qué sigue?

En la [siguiente sesión](https://omarespejel.github.io/espejel_blog/forward/) aprendemos cómo hacer nuestros propias arquitecturas. Las siguientes sesiones dependen en gran medida de lo aprendido en esta entrada por lo que recomiendo entenderla lo más a fondo posible.